In [41]:
"""
    Apartado de importaciones
"""
import pandas as pd
import os

RUTA_DATASET_INICIAL = "datasets/"
RUTA_DATASET_LIMPIADO = "01_dataset_limpiado/"
RUTA_DATASET_FRECUENCIA_GRUPOS = "02_frecuencia_grupos/"
RUTA_DATASET_GRUPOS_CONSOLIDADO = "04_consolidado_grupos/"

COLUMNAS_INICIALES_INTERES = ["358_cause_recode", "sex", "age_recode_52", "race"]
COLUMNAS_GRUPOS_INTERES = ["358_cause_recode", "sex", "age_recode_52", "race", "count", "year"]

NOMBRE_ARCHIVO_CONSOLIDADO_FREC_GRUPO = "frecuencia_grupos_enfermedades.csv"

# Creando directorios en caso no se encuentren en el root
if(not os.path.exists(RUTA_DATASET_INICIAL)): os.makedirs(RUTA_DATASET_INICIAL)
if(not os.path.exists(RUTA_DATASET_LIMPIADO)): os.makedirs(RUTA_DATASET_LIMPIADO)
if(not os.path.exists(RUTA_DATASET_FRECUENCIA_GRUPOS)): os.makedirs(RUTA_DATASET_FRECUENCIA_GRUPOS)
if(not os.path.exists(RUTA_DATASET_GRUPOS_CONSOLIDADO)): os.makedirs(RUTA_DATASET_GRUPOS_CONSOLIDADO)

In [42]:
"""
    01 - Algoritmo de limpiado y extraccion

    Objetivo:
    *) En base a los datasets iniciales, que cuentan con columnas innecesarias que se no se ocuparan en la investigacion
       el algoritmo de limpiado y extraccion busca guardar los datasets, solo con las columnas que se buscan estudiar
"""
carpeta_inicial = RUTA_DATASET_INICIAL
carpeta_destino = RUTA_DATASET_LIMPIADO
columnas_interes = COLUMNAS_INICIALES_INTERES
archivos_convertir = os.listdir(carpeta_inicial)

for archivo in archivos_convertir:
    print("Convirtiendo: " + archivo)
    df_muertes = pd.read_csv(carpeta_inicial + archivo)
    df_resumido = df_muertes[columnas_interes]
    df_resumido.to_csv(carpeta_destino + archivo, index=False)
    print("Dataset almacenado")

Convirtiendo: 2014.csv


C:\Users\JRG\AppData\Local\Temp\ipykernel_25816\3170342760.py:15: DtypeWarning: Columns (39,40,41,42,43,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df_muertes = pd.read_csv(carpeta_inicial + archivo)


Dataset almacenado
Convirtiendo: 2015.csv


C:\Users\JRG\AppData\Local\Temp\ipykernel_25816\3170342760.py:15: DtypeWarning: Columns (40,41,42,43,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df_muertes = pd.read_csv(carpeta_inicial + archivo)


Dataset almacenado


In [43]:
"""
    02 - Algoritmo de agrupamiento

    Objetivo:
    *) En el paso anterior se realizó el proceso de extraccion y limpiado de datos para cada uno de los años 
       que se buscan estudiar, para este paso se busca saber la frecuencia de los grupos de interes, en este
       caso un grupo esta formado por:

        -> Genero
        -> Edad
        -> Raza
        -> Enfermedad
"""

carpeta_inicial = RUTA_DATASET_LIMPIADO
carpeta_destino = RUTA_DATASET_FRECUENCIA_GRUPOS
columnas_interes = COLUMNAS_INICIALES_INTERES
archivos_convertir = os.listdir(carpeta_inicial)

for archivo in archivos_convertir:
    print("Procesando agrupaciones para: " + archivo)
    df_muertes = pd.read_csv(carpeta_inicial + archivo)
    grupo = df_muertes.groupby(columnas_interes).size().sort_values(ascending=False)
    df_agrupado = grupo.reset_index().rename(columns = { 0: "count" }).sort_values(by = columnas_interes)
    # Agregando el año de todas las filas (para distinguir el año de procedencia de la data)
    df_agrupado["year"] = int(archivo.split(".")[0])
    df_agrupado.to_csv(carpeta_destino + archivo, index=False)
    print("Agrupaciones almacenadas")


Procesando agrupaciones para: 2014.csv
Agrupaciones almacenadas
Procesando agrupaciones para: 2015.csv
Agrupaciones almacenadas


In [ ]:
"""
    03 - Algoritmo de estandarización
"""

In [44]:
"""
    04 - Algoritmo de consolidación y agrupación final
"""
carpeta_inicial = RUTA_DATASET_FRECUENCIA_GRUPOS
carpeta_destino = RUTA_DATASET_GRUPOS_CONSOLIDADO
columnas_interes = COLUMNAS_GRUPOS_INTERES
archivos_convertir = os.listdir(carpeta_inicial)

primer_archivo = True
df_grupos_consolidado = ""

for archivo in archivos_convertir:
    if (primer_archivo):
        df_grupos_consolidado = pd.read_csv(carpeta_inicial + archivo)
        primer_archivo = False
    else:
        df_grupo_anio = pd.read_csv(carpeta_inicial + archivo)
        df_grupos_consolidado = pd.concat([df_grupos_consolidado, df_grupo_anio])

# Almacenando consolidado final de enfermedades por anio
df_grupos_consolidado.to_csv(carpeta_destino + NOMBRE_ARCHIVO_CONSOLIDADO_FREC_GRUPO, index=False)
print("Consolidado de frecuencias por enfermedad almacenado exitosamente")

Consolidado de frecuencias por enfermedad almacenado exitosamente
